In [1]:
import os
import boto3
import pandas as pd
import json
from PIL import Image
import io
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
s3 = boto3.client('s3')
bucket_name = 'sagemaker-eu-north-1-495599732227'

In [3]:
class COCODataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['file_name']
        img_path = os.path.join(self.image_dir, img_name)

        # Download image from S3
        img_data = download_file(bucket_name, img_path)
        image = Image.open(io.BytesIO(img_data))

        # Convert to RGB 
        if image.mode != 'RGB':
            image = image.convert('RGB')

        tags = torch.tensor(self.df.iloc[idx]['tags'])
        if self.transform:
            image = self.transform(image)
        return image, tags


In [4]:
def download_file(bucket, key):
    response = s3.get_object(Bucket=bucket, Key=key)
    return response['Body'].read()

In [5]:
def preprocess_images(bucket, image_prefix, annotation_key):
    
    annotations = json.loads(download_file(bucket, annotation_key))

    image_data = []

    for img in tqdm(annotations['images']):
       
        img_data = download_file(bucket, f"{image_prefix}/{img['file_name']}")
        pil_img = Image.open(io.BytesIO(img_data))

        
        pil_img = pil_img.resize((224, 224))

        
        pil_img = pil_img.convert('RGB')  


        
        buffer = io.BytesIO()
        pil_img.save(buffer, format='JPEG')
        s3.put_object(Bucket=bucket, Key=f"preprocessed2/images/{img['file_name']}", Body=buffer.getvalue())

       
        tags = [0] * 80
        for ann in annotations['annotations']:
            if ann['image_id'] == img['id']:
                category_id = ann['category_id'] - 1
                if 0 <= category_id < 80:  
                    tags[category_id] = 1
                else:
                    print(f"Invalid category_id: {category_id}")  

        image_data.append({
            'image_id': img['id'],
            'file_name': img['file_name'],
            'tags': tags
        })

    
    df = pd.DataFrame(image_data)
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)
    s3.put_object(Bucket=bucket, Key='preprocessed2/image_data.csv', Body=csv_buffer.getvalue())

  
    dataset = COCODataset(df, 'preprocessed2')
    torch.save(dataset, 'coco_dataset.pth')
    s3.upload_file('coco_dataset.pth', bucket, 'preprocessed2/coco_dataset.pth')



In [6]:
preprocess_images(bucket_name, 'images', 'annotations/subset_annotations.json')

  0%|          | 2/2000 [00:00<03:42,  9.00it/s]

Invalid category_id: 80
Invalid category_id: 81
Invalid category_id: 80


  0%|          | 7/2000 [00:00<04:05,  8.12it/s]

Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83
Invalid category_id: 83


  2%|▏         | 48/2000 [00:06<04:00,  8.13it/s]

...

 99%|█████████▉| 1987/2000 [04:13<00:01,  8.35it/s]

Invalid category_id: 83


100%|█████████▉| 1998/2000 [04:14<00:00,  6.97it/s]

Invalid category_id: 83
Invalid category_id: 85


100%|██████████| 2000/2000 [04:15<00:00,  7.84it/s]
